In [1]:
import pandas as pd
import xlrd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#return outliers > mean+3sigma or < mean-3sigma
def outlier_3sigma(pd_column):
    mean_vals = pd_column.mean()
    std_vals = pd_column.std()
    min_value = mean_vals-3*std_vals
    max_value = mean_vals+3*std_vals
    outliers = np.nonzero((pd_column.values>max_value)|(pd_column.values<min_value))
    return outliers

In [3]:
def run_columns(pd_column,idx2):
    for col_header in pd_column.columns:
        idx1 = outlier_3sigma(pd_column[col_header])
        idx2 = np.union1d(idx1,idx2)
        idx2 = np.unique(idx2)
    return idx2

## DEFINING VARIABLES 
**df_without_outlier** --- Variable without outlier with 40% refrigerant leaking

**dfNormal_without_outlier** --- Variable without outlier with full charge of refrigerant

In [4]:
path_to_xls = "F:/Talles/Doutorado/database/1043-RP/FDD Data/Refrigerant leak/rl40.xls"
sheet = "Complete Data Set"
leaking_40 = pd.read_excel(path_to_xls,sheet)
idx2 = np.array([])
idx2 = run_columns(leaking_40,idx2)
dfLeaking40_without_outlier= leaking_40.drop(idx2)
path_to_xls = "F:/Talles/Doutorado/database/1043-RP/FDD Data/Benchmark Tests/normal r.xls"
sheet = "Complete Data Set"
Normal_R = pd.read_excel(path_to_xls,sheet)
idx2 = np.array([])
idx2 = run_columns(Normal_R,idx2)
dfNormal_without_outlier = Normal_R.drop(idx2)
# reseting index
dfLeaking40_without_outlier = dfLeaking40_without_outlier.reset_index(drop=True)
dfNormal_without_outlier = dfNormal_without_outlier.reset_index(drop=True)

In [5]:
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler
pca = decomposition.PCA()

In [6]:
print("dataframe dfNormal_without_outlier.shape:{}".format(dfNormal_without_outlier.shape))

st_Normal = StandardScaler().fit_transform(dfNormal_without_outlier)
pca = decomposition.PCA(0.90)
st_Normal90 = pca.fit_transform(st_Normal) # << to retain the components in an object
print(st_Normal90.shape)

dataframe dfNormal_without_outlier.shape:(4663, 66)
(4663, 7)


In [7]:
print("dataframe dfLeaking40_without_outlier.shape:{}".format(dfLeaking40_without_outlier.shape))

st_leaking = StandardScaler().fit_transform(dfLeaking40_without_outlier)
pca = decomposition.PCA(0.90)
st_leaking90 = pca.fit_transform(st_leaking) # << to retain the components in an object
print(st_leaking90.shape)

dataframe dfLeaking40_without_outlier.shape:(4655, 66)
(4655, 7)


## Adding y value on numpy array
0 - Normal <br>
1 - Leaking

In [8]:
print("st_Normal90.shape := {} and st_leaking90.shape := {}".format(st_Normal90.shape,st_leaking90.shape))
st_Normal90 = np.c_[st_Normal90,np.zeros(len(st_Normal90))]
st_leaking90 = np.c_[st_leaking90,np.ones(len(st_leaking90))]
print("adding column y")
print("st_Normal90.shape := {} and st_leaking90.shape := {}".format(st_Normal90.shape,st_leaking90.shape))

st_Normal90.shape := (4663, 7) and st_leaking90.shape := (4655, 7)
adding column y
st_Normal90.shape := (4663, 8) and st_leaking90.shape := (4655, 8)


In [9]:
data = np.vstack((st_Normal90,st_leaking90))
print(data.shape)

(9318, 8)


In [10]:
x,y = (data[:,:-1],data[:,-1])
print(x.shape,y.shape)

(9318, 7) (9318,)


In [11]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

In [12]:
# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(6243, 7) (3075, 7) (6243,) (3075,)


In [13]:
# standardscaler x data
sc_x = StandardScaler()
X_trainScaled = sc_x.fit_transform(X_train)
X_testScaled = sc_x.fit_transform(X_test)

In [14]:
clf = MLPClassifier(hidden_layer_sizes=(256,128,64,32),activation="relu",random_state=1).fit(X_trainScaled, y_train)
y_pred=clf.predict(X_testScaled)
print(clf.score(X_testScaled, y_test))

0.9928455284552845
